먼저, 내가 open3d로 만든 FPFH디스크립터가 어느정도 잘 동작하는지 확인하고 학습해야 할 듯.
내가 잘못 만들어서 문제가 된 것일 수 있으니 확인

In [5]:
import os
import sys
import argparse
import open3d as o3d
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import open3d.visualization as vis
import torch
from scipy.spatial.distance import cdist

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [6]:
opt = argparse.Namespace(
    fpfh_normal_radiuse = 0.3,
    fpfh_descriptors_radiuse = 1.00,
    seq_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    mdgat_path = './KITTI',
    kitti_path = '/media/vision/Seagate/DataSets/kitti/dataset/sequences',
    save_path = './KITTI/keypoints/tsf_256_FPFH_nr05_dr125',
    memory_is_enough = True,
    mutual_check = True,
    transform_opt = 0
)

In [44]:
class mdgat_kitti_data_set():
    def __init__(self, opt):
        self.kitti_path = opt.kitti_path
        self.mdgat_path = opt.mdgat_path
        self.seq_list = opt.seq_list
        self.memory_is_enough = opt.memory_is_enough
        self.mutual_check = opt.mutual_check
        self.transform_opt = opt.transform_opt
        self.fpfh_normal_radiuse = opt.fpfh_normal_radiuse
        self.fpfh_descriptors_radiuse = opt.fpfh_descriptors_radiuse
        self.save_path = opt.save_path

        self.calib = {}
        self.pose = {}
        self.kp = {}
        self.desc = {}
        self.scores = {}
        self.scan = {}
        self.random_sample_num = 16384 #16384
        self.threshold = 0.5

        self._load_preprocessed_data()
    
    def _load_preprocessed_data(self):
        for seq in self.seq_list:
            sequence = '%02d'%seq
            calibpath = os.path.join(self.mdgat_path, 'calib/sequences', sequence, 'calib.txt')
            posepath = os.path.join(self.mdgat_path, 'poses', '%02d.txt'%seq)
            with open(calibpath, 'r') as f:
                for line in f.readlines():
                    _, value = line.split(':', 1)
                    try:
                        calib = np.array([float(x) for x in value.split()])
                    except ValueError:
                        pass
                    calib = np.reshape(calib, (3, 4))    
                    self.calib[sequence] = np.vstack([calib, [0, 0, 0, 1]])
            
            poses = []
            with open(posepath, 'r') as f:
                for line in f.readlines():
                    T_w_cam0 = np.fromstring(line, dtype=float, sep=' ')
                    T_w_cam0 = T_w_cam0.reshape(3, 4)
                    T_w_cam0 = np.vstack((T_w_cam0, [0, 0, 0, 1]))
                    poses.append(T_w_cam0)
                self.pose[sequence] = poses

            '''If memory is enough, load all the data'''
            if self.memory_is_enough:
                kps = []
                scores = []
                desc = []
                folder = os.path.join(self.mdgat_path, 'keypoints/tsf_256_FPFH_16384-512-k1k16-2d-nonoise', sequence)
                folder = os.listdir(folder)   
                folder.sort(key=lambda x:int(x[:-4]))
                for idx in range(len(folder)):
                    file = os.path.join(self.mdgat_path, 'keypoints/tsf_256_FPFH_16384-512-k1k16-2d-nonoise', sequence, folder[idx])
                    if os.path.isfile(file):
                        pc = np.reshape(np.fromfile(file, dtype=np.float32), (-1, 37))
                        ones = np.ones((pc.shape[0], 1))
                        kps.append(np.concatenate((pc[:,:3], ones), axis=1))
                        scores.append(pc[:,3])
                        desc.append(pc[:,4:])
                    else:
                        kps.append([0])

                self.kp[sequence] = kps
                self.desc[sequence] = desc
                self.scores[sequence] = scores

    def _get_kitti_data(self, sequence, index_in_seq):
        pc_file = os.path.join('/media/vision/Seagate/DataSets/kitti/dataset/sequences', sequence, "velodyne" ,'%06d.bin' % index_in_seq)
        pc = np.fromfile(pc_file, dtype=np.float32)
        pc = pc.reshape((-1, 4))

        ones = np.ones((pc.shape[0], 1))
        pc = np.concatenate((pc[:,:3], ones), axis=1)
        # pc = pc[np.random.choice(pc.shape[0], self.random_sample_num, replace=False), :]
        pc = torch.tensor(pc, dtype=torch.double)
        # pc = pc.reshape((-1, 8))
        return pc

    def comute_FPFH(self, pc, kp):
        pcd = o3d.geometry.PointCloud()
        pcd.points = o3d.utility.Vector3dVector(pc[:, :3])

        kp_num = kp.shape[0]
        pcd_kp = o3d.geometry.PointCloud()
        pcd_kp.points = o3d.utility.Vector3dVector(kp[:, :3])
        pcd += pcd_kp

        pcd.estimate_normals(search_param=o3d.geometry.KDTreeSearchParamHybrid(radius=self.fpfh_normal_radiuse, max_nn=30))

        pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(pcd, o3d.geometry.KDTreeSearchParamHybrid(radius=self.fpfh_normal_radiuse, max_nn=100))
        fpfh_desc = np.transpose(pcd_fpfh.data)[-kp_num:].astype(np.float64)
        # fpfh_desc = torch.tensor(fpfh_desc, dtype=torch.double)
        return fpfh_desc

    def test_data(self, seq, idx):
        np.set_printoptions(threshold=np.inf)
        kp = self.kp[seq][idx]
        pc = self._get_kitti_data(seq, idx)
        desc = self.comute_FPFH(pc, kp)
        # print(desc)
        return desc
        
    def save_data(self):
        for seq in self.seq_list:
            sequence = '%02d'%seq
            for idx in range(len(self.kp[sequence])):
                kp = self.kp[sequence][idx]
                pc = self._get_kitti_data(sequence, idx)
                desc = self.comute_FPFH(pc, kp)

                save_keypoints = np.concatenate((kp[:,:3], self.scores[sequence][idx].reshape(-1, 1), desc), axis=1).reshape(-1)
                # print(save_keypoints.shape)

                # Save keypoints as binary file
                os.makedirs(os.path.join(self.save_path, sequence), exist_ok=True) 
                save_file = os.path.join(self.save_path, sequence, '%06d.bin' % idx)
                save_keypoints.tofile(save_file)

                if idx % 100 == 0:
                    print('Sequence %s keypoints saved %d/%d.' % (sequence, idx, len(self.kp[sequence])))
                
            print('Sequence %s keypoints saved.' % sequence)


In [40]:
data_set = mdgat_kitti_data_set(opt=opt)

In [41]:
opt = argparse.Namespace(
    fpfh_normal_radiuse = 0.3,
    fpfh_descriptors_radiuse = 1.00,
    seq_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    mdgat_path = './KITTI',
    kitti_path = '/media/vision/Seagate/DataSets/kitti/dataset/sequences',
    save_path = './KITTI/keypoints/tsf_256_FPFH_16384-512-k1k16-2d-nonoise',
    memory_is_enough = True,
    mutual_check = True,
    transform_opt = 0
)
data = data_set.test_data('00', 0)
print(data)

(array([ 6.10274315, 13.89776802, -0.50446069,  1.        ]), array([5.3948326e+00, 3.2315466e-01, 1.1767133e+00, 4.0945597e+00,
       6.5181923e+00, 7.7943451e+01, 1.7041790e+00, 0.0000000e+00,
       0.0000000e+00, 2.2039515e-01, 2.6245222e+00, 1.8774067e+00,
       1.5169698e+00, 7.9355073e-01, 1.6530827e+00, 1.2222785e+01,
       5.4438622e+01, 1.8504025e+01, 3.5424147e+00, 1.4155250e+00,
       2.0078421e+00, 2.0277805e+00, 4.2965163e-02, 2.0742649e-01,
       5.8658153e-01, 9.4355530e-01, 1.9253677e+01, 3.6063007e+01,
       3.1523884e+01, 5.1284943e+00, 6.3524312e-01, 2.2552669e+00,
       3.3599029e+00], dtype=float32), 0.10588477)


In [42]:
data_set.kp['00'][1].shape

(256, 4)

In [45]:
fpfh_normal_radiuse_list = np.arange(0.05, 0.5, 0.05)
fpfh_descriptors_radiuse_list = np.arange(0.2, 1.5, 0.2)

for fpfh_normal_radiuse in fpfh_normal_radiuse_list:
    for fpfh_descriptors_radiuse in fpfh_descriptors_radiuse_list:
        opt = argparse.Namespace(
            fpfh_normal_radiuse = 0.3,
            fpfh_descriptors_radiuse = 1.00,
            seq_list = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
            mdgat_path = './KITTI',
            kitti_path = '/media/vision/Seagate/DataSets/kitti/dataset/sequences',
            save_path = './KITTI/keypoints/tsf_256_FPFH_nr05_dr125',
            memory_is_enough = True,
            mutual_check = True,
            transform_opt = 0
        )
        data_set = mdgat_kitti_data_set(opt=opt)
        data = data_set.test_data('00', 100)
        print(fpfh_normal_radiuse, fpfh_descriptors_radiuse, data[0])


0.05 0.2 [73.93282972 13.13716064  2.55813028  5.70475429  8.79675042 43.12591369
  7.17965548  6.66486754  6.12994293  3.41400306 29.35599195  9.60900342
  6.35252968  6.69169547 12.58756666 47.39512002 34.64552516 26.25767264
 16.28316868 18.80034437 14.30244635  7.07492755 12.59160001 35.53123974
 39.83835646 22.78970269 11.11785538  6.95097106 14.12310431 13.84304277
 17.83061312 11.72841103 13.65510343]
0.05 0.4 [73.93282972 13.13716064  2.55813028  5.70475429  8.79675042 43.12591369
  7.17965548  6.66486754  6.12994293  3.41400306 29.35599195  9.60900342
  6.35252968  6.69169547 12.58756666 47.39512002 34.64552516 26.25767264
 16.28316868 18.80034437 14.30244635  7.07492755 12.59160001 35.53123974
 39.83835646 22.78970269 11.11785538  6.95097106 14.12310431 13.84304277
 17.83061312 11.72841103 13.65510343]
0.05 0.6000000000000001 [73.93282972 13.13716064  2.55813028  5.70475429  8.79675042 43.12591369
  7.17965548  6.66486754  6.12994293  3.41400306 29.35599195  9.60900342
  6.35

KeyboardInterrupt: 

In [26]:
# data_set.save_data()

Sequence 00 keypoints saved 0/4541.
Sequence 00 keypoints saved 100/4541.
Sequence 00 keypoints saved 200/4541.
Sequence 00 keypoints saved 300/4541.
Sequence 00 keypoints saved 400/4541.
Sequence 00 keypoints saved 500/4541.
Sequence 00 keypoints saved 600/4541.
Sequence 00 keypoints saved 700/4541.
Sequence 00 keypoints saved 800/4541.
Sequence 00 keypoints saved 900/4541.
Sequence 00 keypoints saved 1000/4541.
Sequence 00 keypoints saved 1100/4541.
Sequence 00 keypoints saved 1200/4541.
Sequence 00 keypoints saved 1300/4541.
Sequence 00 keypoints saved 1400/4541.
Sequence 00 keypoints saved 1500/4541.
Sequence 00 keypoints saved 1600/4541.
Sequence 00 keypoints saved 1700/4541.
Sequence 00 keypoints saved 1800/4541.
Sequence 00 keypoints saved 1900/4541.
Sequence 00 keypoints saved 2000/4541.
Sequence 00 keypoints saved 2100/4541.
Sequence 00 keypoints saved 2200/4541.
Sequence 00 keypoints saved 2300/4541.
Sequence 00 keypoints saved 2400/4541.
Sequence 00 keypoints saved 2500/4541

In [23]:
# test save data
sequence = '00'
idx = 0
file = os.path.join(data_set.save_path, sequence, '%06d.bin' % idx)
print(np.fromfile(file, dtype=np.float64).shape)
pc = np.reshape(np.fromfile(file, dtype=np.float64), (-1, 37))
print(pc.shape)
# print(pc[0])


(4736,)
(128, 37)


순서대로 모든 pcw를 합쳐 visualization을 해보자.

In [ ]:
data_set = mdgat_kitti_data_set(opt=opt)
world_pcd = o3d.geometry.PointCloud()
for i in range(0, len(data_set)):
    data = data_set.get_data(0, i)
    pcw = data['cloudw']
    pcdw = o3d.geometry.PointCloud()
    pcdw.points = o3d.utility.Vector3dVector(pcw[:, :3])
    pcdw.voxel_down_sample(voxel_size=0.3)
    world_pcd += pcdw
    if i % 10 == 0:
        print("process %d/%d"%(i, len(data_set)))

vis.draw_geometries([world_pcd])

process 70/4539


역시 딱 맞지 않는다. 

In [ ]:
data_set = mdgat_kitti_data_set(opt=opt)
world_kpd = o3d.geometry.PointCloud()
for i in range(0, len(data_set)):
    data = data_set.get_data(0, i)
    kpw = data['keypointsw']
    kp_pcdw = o3d.geometry.PointCloud()
    kp_pcdw.points = o3d.utility.Vector3dVector(kpw[:, :3])
    kp_pcdw.voxel_down_sample(voxel_size=0.3)
    world_kpd += kp_pcdw
    if i % 10 == 0:
        print("process %d/%d"%(i, len(data_set)))

vis.draw_geometries([world_kpd])

process 0/4539
process 10/4539
process 20/4539
process 30/4539
process 40/4539
process 50/4539
process 60/4539
process 70/4539
process 80/4539
process 90/4539
process 100/4539
process 110/4539
process 120/4539
process 130/4539
process 140/4539
process 150/4539
process 160/4539
process 170/4539
process 180/4539
process 190/4539
process 200/4539
process 210/4539
process 220/4539
process 230/4539
process 240/4539
process 250/4539
process 260/4539
process 270/4539
process 280/4539
process 290/4539
process 300/4539
process 310/4539
process 320/4539
process 330/4539
process 340/4539
process 350/4539
process 360/4539
process 370/4539
process 380/4539
process 390/4539
process 400/4539
process 410/4539
process 420/4539
process 430/4539
process 440/4539
process 450/4539
process 460/4539
process 470/4539
process 480/4539
process 490/4539
process 500/4539
process 510/4539
process 520/4539
process 530/4539
process 540/4539
process 550/4539
process 560/4539
process 570/4539
process 580/4539
process 

In [ ]:
# 비주얼라이제이션
data_set = mdgat_kitti_data_set(opt=opt)
data = data_set.get_data(1100)
coord = 1
if coord == 0:
    pc0 = data['cloud0']
    kp0 = data['keypoints0']
    pc1 = data['cloud1']
    kp1 = data['keypoints1']
else:
    pc0 = data['cloudw0']
    kp0 = data['keypointsw0']
    pc1 = data['cloudw1']
    kp1 = data['keypointsw1']

# Create Open3D point cloud objects
pcd0 = o3d.geometry.PointCloud()
pcd0.points = o3d.utility.Vector3dVector(pc0[:, :3])
pcd0.colors = o3d.utility.Vector3dVector(np.ones((pc0.shape[0], 3)) * [0.5, 0.5, 0.5])

pcd1 = o3d.geometry.PointCloud()
pcd1.points = o3d.utility.Vector3dVector(pc1[:, :3])
pcd1.colors = o3d.utility.Vector3dVector(np.ones((pc1.shape[0], 3)) * [0.2, 0.2, 0.2])

kpcd0 = o3d.geometry.PointCloud()
kpcd0.points = o3d.utility.Vector3dVector(kp0[:, :3])
kpcd0.colors = o3d.utility.Vector3dVector(np.ones((kp0.shape[0], 3)) * [1, 0, 0])

kpcd1 = o3d.geometry.PointCloud()
kpcd1.points = o3d.utility.Vector3dVector(kp1[:, :3])
kpcd1.colors = o3d.utility.Vector3dVector(np.ones((kp1.shape[0], 3)) * [0, 1, 0])

# Visualize the point clouds
vis.draw_geometries([pcd0, kpcd0, pcd1, kpcd1])

만들어진 fpfh값과 precomputed된 fpfh값을 비교해서 어떤 세팅값이 가장 유사한지 찾아보자.

In [ ]:
test_fpfh_normal_radiuses = [0.2, 0.5, 0.8, 1.0, 1.3, 1.5, 1.7]

# test_fpfh_descriptors_radiuses = [0.7 ,1.0, 1.5, 2.0, 2.5]
test_fpfh_descriptors_radiuses = [1.5]

matched_points_rank_mean = np.zeros((len(test_fpfh_normal_radiuses), len(test_fpfh_descriptors_radiuses)))
matched_pointe_dists_mean = np.zeros((len(test_fpfh_normal_radiuses), len(test_fpfh_descriptors_radiuses)))

for j, fpfh_normal_radiuse in enumerate(test_fpfh_normal_radiuses):
    for k, fpfh_descriptors_radiuses in enumerate(test_fpfh_descriptors_radiuses):
        opt = argparse.Namespace(
            fpfh_normal_radiuse = fpfh_normal_radiuse,
            fpfh_descriptors_radiuse = fpfh_descriptors_radiuses,
            seq_list = [0, 2],
            mdgat_path = './KITTI',
            kitti_path = '/media/vision/Seagate/DataSets/kitti/dataset/sequences',
            memory_is_enough = True,
            mutual_check = True,
            transform_opt = 0
        )
        data_set = mdgat_kitti_data_set(opt=opt)

        n = []

        rank_sum = 0
        dist_sum = 0
        ranks = []

        for i in range(100, len(data_set), 500):
            data = data_set.get_data(i)
            match0 = data['gt_matches0']
            desc0 = data['fpfh_descriptors0']
            desc1 = data['fpfh_descriptors1']

            nn = 0
            for idx, matched_idx in enumerate(match0):
                nn += 1
                
                if matched_idx != -1:
                    distance = np.linalg.norm(desc0 - desc1[matched_idx], axis=1)
                    dist_sum += distance[idx]
                    rank_sum += np.argsort(distance).tolist().index(idx)
                    ranks.append(np.argsort(distance).tolist().index(idx))
            n.append(nn)

        matched_points_rank_mean[j,k] = rank_sum / sum(n)
        matched_pointe_dists_mean[j,k] = dist_sum / sum(n)

        print('radius: (', fpfh_normal_radiuse, ', ', fpfh_descriptors_radiuses, ') ', 'rank mean: ', matched_points_rank_mean[j,k], 'dist mean: ', matched_pointe_dists_mean[j,k])
        print('ranks: ', ranks)

만들어진 fpfh가 얼마나 잘 describe 하는지 최적의 세팅값을 찾아보자.

In [ ]:
test_fpfh_normal_radiuses = [0.05, 0.1, 0.2, 0.3, 0.5, 0.8, 1.0, 1.3, 1.5]
test_fpfh_descriptors_radiuses = [0.3, 0.5, 0.7 ,1.0, 1.5, 2.0]

matched_pointe_dists_mean = np.zeros((len(test_fpfh_normal_radiuses), len(test_fpfh_descriptors_radiuses)))

for j, fpfh_normal_radiuse in enumerate(test_fpfh_normal_radiuses):
    for k, fpfh_descriptors_radiuses in enumerate(test_fpfh_descriptors_radiuses):
        opt = argparse.Namespace(
            fpfh_normal_radiuse = fpfh_normal_radiuse,
            fpfh_descriptors_radiuse = fpfh_descriptors_radiuses,
            seq_list = [0, 2],
            mdgat_path = './KITTI',
            kitti_path = '/media/vision/Seagate/DataSets/kitti/dataset/sequences',
            memory_is_enough = True,
            mutual_check = True,
            transform_opt = 0
        )
        data_set = mdgat_kitti_data_set(opt=opt)

        n = []

        rank_sum = 0
        dist_sum = 0

        for i in range(100, len(data_set), 500):
            data = data_set.get_data(i)
            match0 = data['gt_matches0']
            desc0 = data['fpfh_descriptors0']
            desc1 = data['descriptors0']

            nn = 0
            
            for idx in range(len(desc0)):
                nn += 1
                distance = np.linalg.norm(desc0[idx] - desc1[idx])
                dist_sum += distance
            n.append(nn)

        matched_pointe_dists_mean[j,k] = dist_sum / sum(n)

        print('radius: (', fpfh_normal_radiuse, ', ', fpfh_descriptors_radiuses, ') ', 'dist mean: ', matched_pointe_dists_mean[j,k])

In [ ]:
data = data_set.get_data(1100)
# print(data['rep'])
match0 = data['gt_matches0']
match1 = data['gt_matches1']
# desc00 = data['descriptors0']
# desc1 = data['descriptors1']
desc0 = data['fpfh_descriptors0']
desc1 = data['fpfh_descriptors1']
kp0 = data['keypointsw0']
kp1 = data['keypointsw1']
for idx, matched_idx in enumerate(match0):
    if matched_idx != -1:
        print('matched_idx: ', idx, ' & ', matched_idx, ' || distance: ', np.linalg.norm(kp0[idx] - kp1[matched_idx]))
        distance = np.linalg.norm(desc0 - desc1[matched_idx], axis=1)
        print('distance with matched points: ', distance[idx], ' || similarity rank: ', np.argsort(distance).tolist().index(idx), '/', len(distance))
        print('min distance: ', np.min(distance))
        print('mean distance: ', np.mean(distance))
        print('----------------------------------')